# Curious questions to keep studying

What is the average length of the most popular songs (e.g., top 10% popularity)?
Which genres have the fastest 'tempo' and which have the slowest?
Is there any relationship between the popularity of a song and characteristics such as 'valence' (positivity) or 'energy'?
How does acousticness vary between genres? Are there genres that tend to have more acoustic songs?
Can we observe any trends in popularity or other characteristics according to the year of release?

In [1]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import desc, col, isnan, when, count, mean ,stddev, expr

from pyspark.sql.types import StructType, StructField, StringType, DoubleType, DateType, IntegerType

# 1. Create a SparkSession

In [2]:
spark = SparkSession.builder.appName("Spotify Curiosity").getOrCreate()

Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
24/11/14 16:45:59 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [3]:
spark

24/11/14 16:46:13 WARN GarbageCollectionMetrics: To enable non-built-in garbage collector(s) List(G1 Concurrent GC), users should configure it(them) to spark.eventLog.gcMetrics.youngGenerationGarbageCollectors or spark.eventLog.gcMetrics.oldGenerationGarbageCollectors


# 2. Data loading and cleaning¶

In [7]:
# Read CSV file into DataFrame with header
csv_file_path =("./data/spotify_cleaned.csv")
df = spark.read.csv(csv_file_path,header=True,inferSchema=True)

In [8]:
df.printSchema()

root
 |-- name: string (nullable = true)
 |-- artists: string (nullable = true)
 |-- duration_ms: double (nullable = true)
 |-- acousticness: double (nullable = true)
 |-- danceability: double (nullable = true)
 |-- energy: double (nullable = true)
 |-- instrumentalness: double (nullable = true)
 |-- liveness: double (nullable = true)
 |-- loudness: double (nullable = true)
 |-- speechiness: double (nullable = true)
 |-- tempo: double (nullable = true)
 |-- valence: double (nullable = true)
 |-- mode: integer (nullable = true)
 |-- key: integer (nullable = true)
 |-- popularity: integer (nullable = true)
 |-- explicit: integer (nullable = true)
 |-- formatted_date: date (nullable = true)
 |-- tempo_category: string (nullable = true)
 |-- duration_category: string (nullable = true)



In [9]:
popularity_threshold = df.approxQuantile("popularity",[0.9],0.01)[0]

In [ ]:
top_songs = df.filter(col("popularity")>= popularity_threshold)
avg_duration_top_songs = top_songs.selectExpr("avg(duration_ms) as avg_duration").collect()[0]["avg_duration"]
